In [ ]:
%set_env SM_FRAMEWORK=tf.keras

In [ ]:
import import_ipynb
import dataset

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import segmentation_models as sm

In [ ]:
def mask_to_rle(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
model = sm.Unet('efficientnetb2', classes=1, activation='sigmoid')
model.load_weights('models/data_l_100e_b2.h5')

In [ ]:
data_dir = 'data/test_v2/'

file_ds = dataset.image_files(data_dir)
image_ds = dataset.test_dataset(data_dir)

ds = tf.data.Dataset.zip((file_ds, image_ds))

In [ ]:
result_dict = {}

for (file, image) in ds.batch(1).take(10):
    p = model.predict(image)

    for i in range(len(p)):
        dataset.plot_image_mask(image[i], p[i])
        encoded = mask_to_rle(np.round(p[i]))

        f = file[i].numpy().decode('utf-8').split('/')[-1] # get the filename
        result_dict[f] = encoded

In [ ]:
df = pd.DataFrame(list(result_dict.items()), columns=['ImageId', 'EncodedPixels'])
df.to_csv('submission.csv', index=False)